In [56]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras.models as models
import keras.layers as layers
import keras.optimizers as optimizers
import os

In [57]:
data = pd.read_csv(os.getcwd() + '/ratings.sparse.small.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 4499 entries, X1 to X4499
dtypes: int64(4499)
memory usage: 343.2 MB


In [58]:
test = data.sample(frac=0.2)
train = data.drop(test.index)

In [59]:
def build_encoder():
    # encoder
    encoder_input = layers.Input(shape=(4499,))

    x = layers.Dense(90, activation='relu')(encoder_input)
    x = layers.Dense(45, activation='relu')(x)
    encoded = layers.Dense(25, activation='relu')(x)

    # encoder
    return models.Model(encoder_input, encoded)

In [60]:
def build_decoder():
    # decoder
    decoder_input = layers.Input(shape=(25,))

    x = layers.Dense(45, activation='relu')(decoder_input)
    x = layers.Dense(90, activation='relu')(x)
    decoded = layers.Dense(4499)(x)

    # decoder
    return models.Model(decoder_input, decoded)

In [61]:
def masked_mse(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    mask = tf.not_equal(y_true, 0)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.stop_gradient(mask)
    loss = tf.square(y_true - y_pred) * mask
    return tf.reduce_mean(loss)

In [62]:
def build_autoencoder(loss='mean_squared_error'):
    encoder = build_encoder()
    decoder = build_decoder()

    autoencoder_input = layers.Input(shape=(4499,))
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)

    autoencoder = models.Model(autoencoder_input, decoded)
    autoencoder.compile(optimizer='adam', loss=loss)
    return autoencoder

In [63]:
# training
autoencoder = build_autoencoder()
checkpoints = tf.keras.callbacks.ModelCheckpoint('MSE_multiplied.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
with tf.device('/device:GPU:0'):
    history = autoencoder.fit(train, train, epochs=50, batch_size=32, shuffle=True, validation_data=(test, test), workers=2, use_multiprocessing=True, callbacks=[checkpoints])

Epoch 1/50
237/250 [===========================>..] - ETA: 0s - loss: 0.2659
Epoch 1: val_loss improved from inf to 0.24153, saving model to MSE_multiplied.h5
250/250 [==============================] - 3s 10ms/step - loss: 0.2645 - val_loss: 0.2415
Epoch 2/50
246/250 [============================>.] - ETA: 0s - loss: 0.2389
Epoch 2: val_loss improved from 0.24153 to 0.23502, saving model to MSE_multiplied.h5
250/250 [==============================] - 3s 12ms/step - loss: 0.2388 - val_loss: 0.2350
Epoch 3/50
247/250 [============================>.] - ETA: 0s - loss: 0.2324
Epoch 3: val_loss improved from 0.23502 to 0.22811, saving model to MSE_multiplied.h5
250/250 [==============================] - 3s 12ms/step - loss: 0.2327 - val_loss: 0.2281
Epoch 4/50
248/250 [============================>.] - ETA: 0s - loss: 0.2290
Epoch 4: val_loss improved from 0.22811 to 0.22612, saving model to MSE_multiplied.h5
250/250 [==============================] - 3s 11ms/step - loss: 0.2291 - val_loss:

KeyboardInterrupt: 

In [ ]:
# load best model
autoencoder = models.load_model('autoencoder2.h5', custom_objects={'masked_mse': masked_mse})

# plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def test_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    mask = tf.not_equal(y_true, 0)
    mask = tf.cast(mask, dtype=tf.float32)
    # mask /= tf.reduce_mean(mask)
    mask = tf.stop_gradient(mask)
    loss = tf.abs(y_true - y_pred) * mask
    return tf.reduce_mean(loss)